In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep  3 13:49:37 2019

@author: mg21929
"""
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.optimizers import Adam


class patientai:
    '''
    df: Dataframe on which analysis has to be done. It should have only the following columns:

        pat_id_col         : name of the column containing patient ids.

        event_col           : name of the column containing events.

        n_days_from_anchor : name of the column containing number of days the event occured from the anchor date.

        switch_flag        : name of the column containing switch flag.

        cohort             : name of the column containing cohort info.
    '''
    def __init__(self, df, pat_id_col, event_col, n_days_from_anchor, switch_flag, cohort):
        self.cohort = cohort
        self.cohort_name = df[cohort][0:1].tolist()[0]
        self.pat_id_col = pat_id_col
        self.event_col = event_col
        self.n_days_from_anchor = n_days_from_anchor
        self.switch_flag = switch_flag
        self.df = df.drop([cohort] , axis='columns')
        self.df[pat_id_col] = df[pat_id_col].astype(np.int64)
        self.events = list(df[event_col].unique())
        self.events.sort()
        self.max_time_from_anchor = max(df[n_days_from_anchor])



    def get_target(self):

        def func(a_0,a_1):
            if a_0>a_1:
                k = 0
            else:
                k = 1
            return k
        df_temp = self.df.copy()
        df_temp = df_temp.drop([self.event_col,self.n_days_from_anchor] , axis='columns')
        y = df_temp.groupby([self.pat_id_col,self.switch_flag])
        del df_temp
        df_temp = pd.DataFrame(y.size(),columns=['Count']).reset_index()
        df_temp = df_temp.drop(['Count'] , axis='columns')
        df_temp = df_temp.sort_values([self.pat_id_col])
        df_temp.rename(columns = {self.switch_flag:'switch_flag',self.pat_id_col:'pat_id'}, inplace = True)
        return df_temp

    def get_crossectional_data(self, marker_time):
        '''
        marker_time: time in days at which a marker should be placed in the patient journey
        '''
        self.marker_time = marker_time
        df_temp = self.df.copy()
        df_temp = df_temp.drop([self.switch_flag] , axis='columns')
        df_temp = df_temp.sort_values([self.pat_id_col,self.n_days_from_anchor])
        grps = df_temp.groupby([self.pat_id_col])
        journey_list=[]
        time_list=[]
        pat_id_list=[]
        for grp in grps:
            journey_list.append(grp[1][self.event_col].tolist())
            time_list.append(grp[1][self.n_days_from_anchor].tolist())
            pat_id_list.append(grp[0])
        df_temp=pd.DataFrame(columns = ['pat_id','journey','event_day'])
        df_temp['pat_id']=pat_id_list
        df_temp['journey']=journey_list
        df_temp['event_day']=time_list

        def get_marker_indices(days,l):
            indices=[]
            for i in range(len(l)-1):
                if l[i+1]//days != l[i]//days:
                    indices.append(i+1)
            return indices

        def add_markers(indices,lst):
            l = lst.copy()
            k=0
            for i in range(len(indices)-1):
                l.insert(indices[i]+k,'marker')
                k=k+1
            return l

        def add_markers_2(journey, days, marker_time):
            marked_journey = []
            k = 0
            for i in range(len(journey)):
                delta = days[i]-k
                k = days[i]
                if delta>= marker_time:
                    marker_lst = ['marker']*int(delta/marker_time)
                    marked_journey = marked_journey + marker_lst
                    marked_journey.append(journey[i])
                else:
                    marked_journey.append(journey[i])
            return marked_journey

        df_temp['indices'] = df_temp.apply(lambda x: get_marker_indices(self.marker_time,x['event_day']),axis=1)
        df_temp['marked_journey'] = df_temp.apply(lambda x: add_markers(x['indices'],x['journey']),axis=1)
        df_temp['marked_journey_2'] = df_temp.apply(lambda x: add_markers_2(x['journey'],x['event_day'],self.marker_time),axis=1)

        return df_temp

    @staticmethod
    def train_word2vec(corpus, size = 200, epochs = 50, train_incrementally=False):
        """
        corpus is a list of lists with each list being a tokenized sentence.
        If training incrementally size is fixed to 200
        """

        if train_incrementally:
            data= corpus
            model_new = Word2Vec(min_count=1,size=size,window=4, iter = epochs)# iter == epochs
            model_new.build_vocab(data)
            model_new.intersect_word2vec_format('pubmed2018_w2v_200D.bin',binary=True,lockf=0.0)
            model_new.train(data,total_words=model_new.corpus_count,epochs=model_new.epochs)
        else:
            data= corpus
            model_new = Word2Vec(min_count=1,size=size,window=4, iter = epochs)# iter == epochs
            model_new.build_vocab(data)
            model_new.train(data,total_words=model_new.corpus_count,epochs=model_new.epochs)
        return model_new

    @staticmethod
    def get_deepr_tensor(marked_journey_list, loaded_word2vec, n_events):
        '''
        marked_journey_list : list of events

        loaded_word2vec     : word2vec model trained on the corpus containing the events present in the journey

        n_events            : Number of events from the anchor date to be considered for making the image
        '''
        len_journey = len(marked_journey_list)
        word2vec_dim = loaded_word2vec.vector_size
        if len_journey<n_events:
            a=loaded_word2vec.wv[marked_journey_list].T
            zeros = np.zeros((word2vec_dim,(n_events-len_journey)))
            arr = np.concatenate((a,zeros),axis = 1)
        else:
            arr = loaded_word2vec.wv[marked_journey_list[:n_events]].T
        return arr[:,:,np.newaxis]

    @staticmethod
    def get_deepr_batch(df_crossectional, col_name, df_target, switch_flag_col_name, pat_id_col_name_target, batch_size, loaded_word2vec, n_events):
        """
        df_crossectional should have a column 'pat_id' and df_target should have columns
        named 'switch_flag & pat_id'.The columns will be present by default if dataframes are generated
        by using methods of ProcessCohortData class.

        df_crossectional    : Dataframe containing the journey as one of the columns

        col_name            : Name of the column containing the patient journey

        df_target           : Dataframe containing switch flag

        loaded_word2vec     : word2vec model trained on the corpus containing the events present in the journey

        n_events            : Number of events from the anchor date to be considered for making the image
        """
        pat_ids = np.random.choice(df_crossectional['pat_id'],batch_size, replace=False)
        y = np.array(df_target[df_target[pat_id_col_name_target].isin(pat_ids)][switch_flag_col_name])
        x = df_crossectional[df_crossectional['pat_id'].isin(pat_ids)]
        Journey_lists = x[col_name].tolist()
        arr_list = []
        for i in range(len(Journey_lists)):
            arr_list.append(patientai.get_deepr_tensor(Journey_lists[i], loaded_word2vec, n_events))
        return np.array(arr_list),y,x

    @staticmethod
    def get_patimg2d_tensor(event_dict,t_max,journey_list,journey_event_time_list,agg_level = 14, show_image = False):
        p = [journey_list,journey_event_time_list]
        array = np.zeros((t_max, len(event_dict)))

        for i in range(len(p[0])):
            event = p[0][i]
            time = p[1][i]
            array[time-1][event_dict[event]] = 1

        n_iter = t_max/agg_level
        if t_max%agg_level != 0:
            img = np.zeros((int(n_iter)+1,np.shape(array)[1]))
            for i in range(int(n_iter)):
                img[i] = np.sum(array[i*agg_level:(i+1)*agg_level],axis=0)
            img[i+1] = np.sum(array[(i+1)*agg_level:],axis=0)
        else:
            img = np.zeros((int(n_iter),np.shape(array)[1]))
            for i in range(int(n_iter)):
                img[i] = np.sum(array[i*agg_level:(i+1)*agg_level],axis=0)

        if show_image:
            plt.figure()
            plt.imshow(img)
            plt.show()
        return array, img[:,:,np.newaxis]

    @staticmethod
    def get_patimg2d_batch(df_crossectional, col_name, event_day_col, df_target, switch_flag_col_name, pat_id_col_name_target, batch_size,event_dict,t_max,agg_level=14):
        """
        df_crossectional should have a column 'pat_id' and df_target should have columns
        named 'switch_flag & pat_id'.The columns will be present by default if dataframes are generated
        by using methods of ProcessCohortData class.

        df_crossectional    : Dataframe containing the journey as one of the columns

        col_name            : Name of the column containing the patient journey

        event_day_col       : Name of the column containing journey events time from the anchor date for each patient

        df_target           : Dataframe containing switch flag

        event_dict         : Dictionary mapping events to the columns

        agg_level           : Aggregation level in days

        t_max               : Max time in days from the anchor date
        """
        pat_ids = np.random.choice(df_crossectional['pat_id'],batch_size, replace=False)
        y = np.array(df_target[df_target[pat_id_col_name_target].isin(pat_ids)][switch_flag_col_name])
        x = df_crossectional[df_crossectional['pat_id'].isin(pat_ids)]
        Journey_lists = x[col_name].tolist()
        events_day_lists = x[event_day_col].tolist()

        arr_list = []
        for i in range(len(Journey_lists)):
            arr = patientai.get_patimg2d_tensor(event_dict,t_max,Journey_lists[i],events_day_lists[i],agg_level)[1]
            arr_list.append(arr)
        return np.array(arr_list),y,pat_ids

    @staticmethod
    def get_patimg3d_tensor(event_dict, t_max, journey_list, journey_event_time_list, word2vec, agg_level = 14):
        word2vec_dim = word2vec.vector_size
        p = [journey_list,journey_event_time_list]
        array = np.zeros((t_max, len(event_dict), word2vec_dim))

        for i in range(len(p[0])):
            event = p[0][i]
            time = p[1][i]
            array[time-1 , event_dict[event]] = word2vec[event]

        n_iter = t_max/agg_level
        if t_max%agg_level != 0:
            img_agg = np.zeros((int(n_iter)+1,array.shape[1],array.shape[2]))
            for i in range(int(n_iter)):
                img_agg[i] = np.sum(array[i*agg_level:(i+1)*agg_level],axis=0)
            img_agg[i+1] = np.sum(array[(i+1)*agg_level:],axis=0)
        else:
            img_agg = np.zeros((int(n_iter),array.shape[1],array.shape[2]))
            for i in range(int(n_iter)):
                img_agg[i] = np.sum(array[i*agg_level:(i+1)*agg_level],axis=0)
        return array, img_agg

    @staticmethod
    def get_patimg3d_batch(event_dict, df_crossectional, col_name_journey, col_name_days, df_target, switch_flag_col_name, pat_id_col_name_target, 
                              batch_size, loaded_word2vec, t_max, n_events, agg_level = 14):

        pat_ids = np.random.choice(df_crossectional['pat_id'],batch_size, replace=False)
        y = np.array(df_target[df_target[pat_id_col_name_target].isin(pat_ids)][switch_flag_col_name])
        x = df_crossectional[df_crossectional['pat_id'].isin(pat_ids)]
        Journey_lists = x[col_name_journey].tolist()
        Journey_days = x[col_name_days].tolist()
        arr_list = []
        for i in range(len(Journey_lists)):
            arr_list.append(patientai.get_patimg3d_tensor(event_dict, t_max, Journey_lists[i], Journey_days[i], loaded_word2vec,
                                           agg_level=agg_level)[1])

        return np.array(arr_list),y,x

    @staticmethod
    def build_model_patimg2d(input_shape = (53,248,1), n_kernels = 64, kernel_size = (1,248), kernel_strides = (1,1), max_pool_from = (3,1),d_units = 512, lr_optimizer = 0.001, n_static = None, show_summary = True, visualize_model = False):
        """
        Architechture taken  : Conv2D(relu) --> Maxpooling2D --> Flatten --> Dense(relu) --> O/P(sigmoid)

        input_shape          : Shape of the input tensor with channels last

        n_kernels            : No. of kernels in Conv2D

        kernel_size          : Kernel size in Conv2D

        kernel_strides       : Kernel strides in Conv2D

        max_pool_from        : Max pooling window in MaxPooling2D

        d_units              : No. of units in the dense layer

        lr                   : Learning rate of Adam optimizer
        """
        if not n_static:
            input_layer = Input(shape = input_shape)
            l_1 = Conv2D(n_kernels, kernel_size,strides=kernel_strides,activation='relu')(input_layer)
            l_2 = MaxPooling2D(max_pool_from)(l_1)
            l_3 = Flatten()(l_2)
            l_4 = Dense(d_units, activation = 'relu')(l_3)
            l_5 = Dense(1,activation='sigmoid')(l_4)

            model = Model(input_layer,l_5)

            optimizer = Adam(lr=lr_optimizer)
            model.compile(loss='binary_crossentropy', optimizer=optimizer)
        else:
            input_layer = Input(shape = input_shape)
            input_static = Input(shape= (n_static,))
            l_1 = Conv2D(n_kernels, kernel_size,strides=kernel_strides,activation='relu')(input_layer)
            l_2 = MaxPooling2D(max_pool_from)(l_1)
            l_3 = Flatten()(l_2)
            with_static_features = concatenate([l_3 , input_static])
            l_4 = Dense(d_units, activation = 'relu')(with_static_features)
            out = Dense(1,activation='sigmoid')(l_4)

            model = Model(inputs = [input_layer , input_static], outputs = out)

            optimizer = Adam(lr=lr_optimizer)
            model.compile(loss='binary_crossentropy', optimizer=optimizer)

        if show_summary:
            model.summary()
        if visualize_model:
            tf.keras.utils.plot_model(model, show_shapes=True)
        return model

    @staticmethod
    def build_model_deepr(input_shape = (100,190,1), n_kernels = 64, kernel_size = (100,3), kernel_strides = (1,1), max_pool_from = (1,3),d_units = 512, lr_optimizer = 0.001, n_static = None, show_summary = True, visualize_model = False):
        """
        Architechture taken  : Conv2D(relu) --> Maxpooling2D --> Dropout(0.3) --> Flatten --> Dropout(0.4) --> Dense(relu) --> O/P(sigmoid)

        input_shape          : Shape of the input tensor with channels last (embedding_dims, events_taken, channels)

        n_kernels            : No. of kernels in Conv2D

        kernel_size          : Kernel size in Conv2D (word2vec_dim, events_to_look_at)

        kernel_strides       : Kernel strides in Conv2D

        max_pool_from        : Max pooling window in MaxPooling2D

        d_units              : No. of units in the dense layer

        lr                   : Learning rate of Adam optimizer
        """
        if not n_static:
            input_layer = Input(shape = input_shape)
            layer_1 = Conv2D(n_kernels, kernel_size, strides = kernel_strides, activation = 'relu')(input_layer)
            layer_2 = MaxPooling2D(max_pool_from)(layer_1)
            dropout_1 = Dropout(0.3)(layer_2)
            layer_3 = Flatten()(dropout_1)
            dropout_2 = Dropout(0.4)(layer_3)
            layer_4 = Dense(d_units, activation = 'relu')(dropout_2)
            out = Dense(1, activation = 'sigmoid')(layer_4)

            model = Model(input_layer,out)

            optimizer = Adam(lr=lr_optimizer)
            model.compile(loss='binary_crossentropy', optimizer=optimizer)
        else:
            input_static = Input(shape= (n_static,))
            input_layer = Input(shape = input_shape)
            layer_1 = Conv2D(n_kernels, kernel_size, strides = kernel_strides, activation = 'relu')(input_layer)
            layer_2 = MaxPooling2D(max_pool_from)(layer_1)
            dropout_1 = Dropout(0.3)(layer_2)
            layer_3 = Flatten()(dropout_1)
            dropout_2 = Dropout(0.4)(layer_3)
            with_static_features = concatenate([dropout_2 , input_static])
            layer_4 = Dense(d_units, activation = 'relu')(with_static_features)
            out = Dense(1, activation = 'sigmoid')(layer_4)

            model = Model(inputs = [input_layer , input_static], outputs = out)

            optimizer = Adam(lr=lr_optimizer)
            model.compile(loss='binary_crossentropy', optimizer=optimizer)

        if show_summary:
            model.summary()
        if visualize_model:
            tf.keras.utils.plot_model(model, show_shapes=True)
        return model

    @staticmethod
    def build_model_patimg3d(input_shape = (53,248,50), n_kernels = 64, kernel_size = (1,248), kernel_strides = (1,1), max_pool_from = (3,1),d_units = 512, lr_optimizer = 0.001, n_static = None, show_summary = True, visualize_model = False):
        """
        Architechture taken  : Conv2D(relu) --> Maxpooling2D --> Flatten --> Dense(relu) --> O/P(sigmoid)

        input_shape          : Shape of the input tensor with channels last (time, n_events, embedding_dims)

        n_kernels            : No. of kernels in Conv2D

        kernel_size          : Kernel size in Conv2D

        kernel_strides       : Kernel strides in Conv2D

        max_pool_from        : Max pooling window in MaxPooling2D

        d_units              : No. of units in the dense layer

        lr                   : Learning rate of Adam optimizer
        """
        if not n_static:
            input_layer = Input(shape = input_shape)
            layer_1 = Conv2D(n_kernels,kernel_size, strides = kernel_strides, activation = 'relu')(input_layer)
            layer_2 = MaxPooling2D(max_pool_from)(layer_1)
            layer_3 = Flatten()(layer_2)
            layer_4 = Dense(d_units, activation = 'relu')(layer_3)
            out = Dense(1, activation = 'sigmoid')(layer_4)

            model = Model(input_layer,out)

            optimizer = Adam(lr=lr_optimizer)
            model.compile(loss='binary_crossentropy', optimizer=optimizer)
        else:
            input_static = Input(shape= (n_static,))
            input_layer = Input(shape = input_shape)
            layer_1 = Conv2D(n_kernels,kernel_size, strides = kernel_strides, activation = 'relu')(input_layer)
            layer_2 = MaxPooling2D(max_pool_from)(layer_1)
            layer_3 = Flatten()(layer_2)
            with_static_features = concatenate([layer_3 , input_static])
            layer_4 = Dense(d_units, activation = 'relu')(with_static_features)
            out = Dense(1, activation = 'sigmoid')(layer_4)

            model = Model(inputs = [input_layer , input_static], outputs = out)

            optimizer = Adam(lr=lr_optimizer)
            model.compile(loss='binary_crossentropy', optimizer=optimizer)

        if show_summary:
            model.summary()
        if visualize_model:
            tf.keras.utils.plot_model(model, show_shapes=True)
        return model

    @staticmethod
    def deepr_generator(df_crossectional, journey_col_name, df_target, switch_flag_col_name, loaded_word2vec, n_events,
                        batch_size = 64, iterate_indefinitely = True):
        n_iter = int(len(df_crossectional)/batch_size)
        i = 0
        while True:
            inital_index = batch_size*i
            final_index = batch_size*(i+1)
            i += 1

            Journey_lists = df_crossectional[journey_col_name][inital_index:final_index].tolist()
            arr_list = []
            for journey in Journey_lists:
                arr_list.append(patientai.get_deepr_tensor(journey,loaded_word2vec,n_events))
            x = np.array(arr_list)
            y = np.array(df_target[switch_flag_col_name][inital_index:final_index])

            if iterate_indefinitely and i == n_iter:
                i = 0
            if not iterate_indefinitely and i == n_iter:
                yield x, y
                break
            yield x,y

    @staticmethod
    def patimg2d_generator(df_crossectional, journey_col_name, event_day_col_name, df_target, switch_flag_col_name, event_dict,
                           t_max, agg_level = 14, batch_size = 64, iterate_indefinitely = True):

        n_iter = int(len(df_crossectional)/batch_size)
        i = 0
        while True:
            inital_index = batch_size*i
            final_index = batch_size*(i+1)
            i += 1

            Journey_lists = df_crossectional[journey_col_name][inital_index:final_index].tolist()
            event_day_lists = df_crossectional[event_day_col_name][inital_index:final_index].tolist()
            arr_list = []
            for k in range(len(Journey_lists)):
                img_agg = patientai.get_patimg2d_tensor(event_dict,t_max,Journey_lists[k],event_day_lists[k],agg_level)[1]
                arr_list.append(img_agg)
            x = np.array(arr_list)
            y = np.array(df_target[switch_flag_col_name][inital_index:final_index])

            if iterate_indefinitely and i == n_iter:
                i = 0
            if not iterate_indefinitely and i == n_iter:
                yield x, y
                break
            yield x, y

    @staticmethod
    def patimg3d_generator(df_crossectional, journey_col_name, event_day_col_name, df_target, switch_flag_col_name, event_dict,
                           t_max, loaded_word2vec, agg_level = 14, batch_size = 64, iterate_indefinitely = True):

        n_iter = int(len(df_crossectional)/batch_size)
        i = 0
        while True:
            inital_index = batch_size*i
            final_index = batch_size*(i+1)
            i += 1

            Journey_lists = df_crossectional[journey_col_name][inital_index:final_index].tolist()
            event_day_lists = df_crossectional[event_day_col_name][inital_index:final_index].tolist()
            arr_list = []
            for k in range(len(Journey_lists)):
                img_agg = patientai.get_patimg3d_tensor(event_dict,t_max,Journey_lists[k],event_day_lists[k],loaded_word2vec, agg_level)[1]
                arr_list.append(img_agg)
            x = np.array(arr_list)
            y = np.array(df_target[switch_flag_col_name][inital_index:final_index])

            if iterate_indefinitely and i == n_iter:
                i = 0
            if not iterate_indefinitely and i == n_iter:
                yield x, y
                break
            yield x, y


/databricks/python/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
 _deprecated()

In [2]:
import sys
import pandas as pd 
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics import roc_auc_score, precision_recall_curve  
from tqdm import tqdm_notebook as tqdm
from tensorflow.keras.models import model_from_json
import matplotlib.pyplot as plt

In [3]:
# bucket_path = 's3a://amgen-edl-gco-us-analytics-repai-development/treatment_pattern/refreshed_tables/data_cohorts/'
# local_path = 'databricks/driver/'

# def copy_to_s3(local_path, bucket_path, file_name_with_extension):
#   dbutils.fs.cp("file:/"+local_path+file_name_with_extension, bucket_path+file_name_with_extension)
#   print('Copied')

# def copy_from_s3(local_path, bucket_path, file_name_with_extension):
#   dbutils.fs.cp(bucket_path+file_name_with_extension, "file:/"+local_path+file_name_with_extension)
#   print('Copied')

In [4]:
# copy_from_s3(local_path, bucket_path, 'training_cohort_6mo_till_Oct.csv')
# train_data_pd = pd.read_csv('training_cohort_6mo_till_Oct.csv')

Copied

In [5]:
train_data = spark.sql('select * from training_data_for_cnn_no_exclusion')
# validation_data = spark.sql('select * from validation_data_for_cnn')

In [6]:
validation_data = spark.sql('select * from validation_data_for_cnn_no_exclusion')

In [7]:
train_data_pd.to_pickle('train.pkl')

In [8]:
val_data_pd.to_pickle('test.pkl')

In [9]:
events_chi2_p_value = spark.sql('select * from events_chi2_p_value')

In [10]:
events_chi2_p_value = events_chi2_p_value.toPandas()

In [11]:
events_chi2_p_value.to_pickle("events_chi2_p_value.pkl")

In [12]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [13]:
train_data_pd = train_data.toPandas()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [14]:
val_data_pd = validation_data.toPandas()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [15]:
df_dict = dict()
df_dict['RX'] = train_data_pd[train_data_pd['event_type']=='RX'].reset_index(drop=True)
df_dict['PX'] = train_data_pd[train_data_pd['event_type']=='PX'].reset_index(drop=True)
df_dict['DX'] = train_data_pd[train_data_pd['event_type']=='DX'].reset_index(drop=True)

In [16]:
cols_to_keep = ['patient_id', 'event_date', 'days_from_anchor', 'event_code', 'switch_flag']

train_pd = train_data_pd[cols_to_keep].copy(deep=True)
# val_pd = val_data_pd[cols_to_keep].copy(deep=True)

In [17]:
for event_type in ['RX', 'PX', 'DX']:
  df_dict[event_type]['cohort'] = 'c_1'
  df_dict[event_type] = patientai(df_dict[event_type],'patient_id','event_code','days_from_anchor','switch_flag','cohort')
  df_dict[event_type+'_cross'] = df_dict[event_type].get_crossectional_data(30)
  df_dict[event_type+'_y'] = df_dict[event_type].get_target()

In [18]:
for data_name in ['RX_cross', 'RX_y', 'PX_cross', 'PX_y', 'DX_cross', 'DX_y']:
  df_dict[data_name].to_pickle(data_name+'_eclong.pkl')

In [19]:
df_dict = dict()
for data_name in ['RX_cross', 'RX_y', 'PX_cross', 'PX_y', 'DX_cross', 'DX_y']:
  df_dict[data_name] = pd.read_pickle(data_name+'_eclong.pkl')

In [20]:
def event_count(row, ndays):
  count_dict = dict()
  for ind, day in enumerate(list(row['event_day'])):
    if day >= ndays:
      break
  max_ind = ind
  for event in set(row['journey']):
    count_dict[event]=row['journey'][:max_ind].count(event)
  
  return count_dict

def get_event_frequency_df(df_cross_sec, n_days):
  df_cross_sec['event_freq_dict'] = df_cross_sec.apply(event_count, axis=1, args=(n_days,))
  patient_event_frequency_dict = dict(zip(df_cross_sec['pat_id'].tolist(), df_cross_sec['event_freq_dict'].tolist()))
  freq_df = pd.DataFrame.from_dict(patient_event_frequency_dict, orient='index')
  freq_df.fillna(0, inplace=True)
  
  return freq_df

In [21]:
'RX_cross'[:2]+'_freq'

Out[11]: 'RX_freq'

In [22]:
n_days = 730
for data_name in ['RX_cross', 'PX_cross', 'DX_cross']:
  df_dict[data_name[:2]+'_freq'] = get_event_frequency_df(df_dict[data_name], n_days)
  df_dict[data_name[:2]+'_freq'] = df_dict[data_name[:2]+'_freq'].reset_index().rename(columns={'index':'patient_id'})
  df_dict[data_name[:2]+'_freq'] = df_dict[data_name[:2]+'_freq'].sort_values('patient_id').reset_index(drop=True)
  df_dict[data_name[:2]+'_y'] = df_dict[data_name[:2]+'_y'].sort_values('pat_id').reset_index(drop=True)

In [23]:
from sklearn.linear_model import LogisticRegression
from scipy.sparse import csr_matrix

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [24]:
X = np.array(df_dict['DX_freq'].iloc[:,1:])
y = np.array(df_dict['DX_y']['switch_flag'])

chi2_scores, p_values = chi2(X, y)
t = pd.DataFrame({'event_code': df_dict['DX_freq'].columns[1:], 'chi2': chi2_scores, 'p_value': p_values})
feature_importance_dx = t.sort_values('chi2', ascending=False).reset_index(drop=True)

In [25]:
display(feature_importance_dx)

event_code,chi2,p_value
I25.10,1777.28233664736,0.0
I21.02,769.0606055708336,2.8774532123459093E-169
I25.110,698.7161515620907,5.687033648955905E-154
E11.9,687.6292334380738,1.4650173042366357E-151
I70.719,623.3344881455766,1.407736065435451E-137
I21.4,530.8333152521557,1.860616397603256E-117
I20.0,527.8343676250391,8.357871617075028E-117
M62.82,462.954792819011,1.093658526535715E-102
I25.2,413.240548295131,7.223522340529144E-92
I25.119,360.80075800934003,1.8846110723374788E-80


In [26]:
X = np.array(df_dict['RX_freq'].iloc[:,1:])
y = np.array(df_dict['RX_y']['switch_flag'])

chi2_scores, p_values = chi2(X, y)
t = pd.DataFrame({'event_code': df_dict['RX_freq'].columns[1:], 'chi2': chi2_scores, 'p_value': p_values})
feature_importance_rx = t.sort_values('chi2', ascending=False).reset_index(drop=True)

In [27]:
display(feature_importance_rx)

event_code,chi2,p_value
ATORVASTATIN CALCIUM,770.485747659541,1.4097484116773755E-169
EZETIMIBE-SIMVASTATIN,558.1973015950584,2.073803823489759E-123
LIVALO,538.907513330705,3.259125765504011E-119
CLOPIDOGREL,371.8435675535663,7.426684349520426E-83
PRAVASTATIN SODIUM,305.7769876887073,1.8163501344202924E-68
EZETIMIBE,298.4755436157478,7.077858015696315E-67
ROSUVASTATIN CALCIUM,268.5773810086883,2.3172123816581762E-60
CRESTOR,157.30010413313812,4.4012250492738894E-36
VASCEPA,109.76988871960287,1.10052488529042E-25
LIPITOR,107.39091079464839,3.654789326823154E-25


In [28]:
X = np.array(df_dict['PX_freq'].iloc[:,1:])
y = np.array(df_dict['PX_y']['switch_flag'])

chi2_scores, p_values = chi2(X, y)
t = pd.DataFrame({'event_code': df_dict['PX_freq'].columns[1:], 'chi2': chi2_scores, 'p_value': p_values})
feature_importance_px = t.sort_values('chi2', ascending=False).reset_index(drop=True)

In [29]:
display(feature_importance_px)

event_code,chi2,p_value
33519,14.565608120425114,1.3536268226075747E-4
92973,10.7738880961895,0.0010294203736134994
92944,10.742729793740002,0.0010468979040132215
33521,10.11313552334392,0.0014721628823706227
37228,9.993359618178047,0.0015710571463311012
37225,8.704372968714079,0.0031744759120255314
92943,6.500628843414415,0.010783634557551233
33512,3.591296032063166,0.05808292833919944
37226,3.2256509759641765,0.0724929684460467
37229,3.054181622674709,0.08052955442894559


In [30]:
feature_importance_rx.shape

Out[18]: (56, 3)

In [31]:
imp_rx = feature_importance_rx[feature_importance_rx['p_value']<=0.05]['event_code'].tolist()
imp_px = feature_importance_px[feature_importance_px['p_value']<=0.05]['event_code'].tolist()
imp_dx = feature_importance_dx[feature_importance_dx['p_value']<=0.05]['event_code'].tolist()
all_imp_events = imp_rx + imp_px + imp_dx
len(all_imp_events)

Out[19]: 196

In [32]:
temp_df = pd.DataFrame(all_imp_events)
temp_df.rename(columns = {0:'imp_events'},inplace=True)

In [33]:
temp_df = spark.createDataFrame(temp_df)

temp_df.createOrReplaceTempView('temp_df_view')

In [34]:
%sql
drop table if exists events_chi2_p_value;

create table events_chi2_p_value as
select * from temp_df_view;

In [35]:
training_data_for_cnn_no_exclusion

In [36]:
%sql 
describe events_chi2_p_value

col_name,data_type,comment
imp_events,string,null


In [37]:
temp_df.to_pickle('chi2_p_value_events_6mo_till_Oct.pkl')

In [38]:
bucket_path = 's3a://amgen-edl-gco-us-analytics-repai-development/treatment_pattern/refreshed_tables/chi2_pvalue/'
local_path = 'databricks/driver/'

def copy_to_s3(local_path, bucket_path, file_name_with_extension):
  dbutils.fs.cp("file:/"+local_path+file_name_with_extension, bucket_path+file_name_with_extension)
  print('Copied')

def copy_from_s3(local_path, bucket_path, file_name_with_extension):
  dbutils.fs.cp(bucket_path+file_name_with_extension, "file:/"+local_path+file_name_with_extension)
  print('Copied')
  
copy_to_s3(local_path, bucket_path, 'chi2_p_value_events_6mo_till_Oct.pkl')

Copied

In [39]:
%sh ls

adstock_scores_p_value_events.pkl
adstock_with_hml.pkl
chi2_p_value_events_1year_till_Oct.pkl
chi2_p_value_events_1yr_till_Oct.pkl
chi2_p_value_events_2years_till_Nov.pkl
chi2_p_value_events_2yrs_till_Oct.pkl
chi2_p_value_events_6mo_till_Oct.pkl
conf
derby.log
DX_cross_eclong.pkl
DX_y_eclong.pkl
eventlogs
ganglia
logs
PX_cross_eclong.pkl
PX_y_eclong.pkl
RX_cross_eclong.pkl
RX_y_eclong.pkl
stacked_lstm_128_w2v32_08-0.86.h5
test_cross_data_chi2_pvalue_refreshed_tables_modified.pkl
test_cross_data_chi2_pvalue_refreshed_tables.pkl
test_cross_data_refreshed_tables.pkl
train_cross_data_chi2_pvalue_refreshed_tables_modified.pkl
train_cross_data_chi2_pvalue_refreshed_tables.pkl
train_cross_data_refreshed_tables.pkl
training_cohort_1yr_till_Oct.csv
training_cohort_2yrs_till_Oct.csv
training_cohort_6mo_till_Oct.csv
w2v_32_refreshed_tables.model

In [40]:
classifier = LogisticRegression(random_state=99, solver='saga', max_iter=2000)
classifier.fit(x, y)

In [41]:
def select_features(df_X, df_y, frac_num_features):
  skbest = SelectKBest(chi2, k=int(frac_num_features*len(df_X.columns[1:])))
  X = skbest.fit_transform(df_X[df_X.columns[1:]], np.array(df_y['switch_flag']))
  keep_features = dict(zip(df_X.columns[1:] ,list(skbest.get_support())))
  keep_features_list = [key for key in keep_features.keys() if keep_features[key]==True]
  X_df = pd.DataFrame(X, columns = keep_features_list)
  
  X_df_sparse = csr_matrix(X_df)
  
#   print('Fitting Logistic Regression')
#   fit_logistic_reg(X_df_sparse, np.array(df_y['switch_flag']))
#   preds = clf.predict_proba(X_df_sparse)[:,1]
#   print(roc_auc_score(np.array(df_y['switch_flag']), preds))
  
  return X_df

In [42]:
dx_feat = select_features(df_dict['DX_freq'], df_dict['DX_y'], 0.6)

In [43]:
rx_feat = select_features(df_dict['RX_freq'], df_dict['RX_y'], 0.6)

In [44]:
rx_cols_renamed = []
for rx_code in rx_feat.columns:
  new_name = '_'.join([elem for elem in rx_code.split()])
  rx_cols_renamed.append(new_name)

rx_feat.columns = rx_cols_renamed

In [45]:
px_feat = select_features(df_dict['PX_freq'], df_dict['PX_y'], 0.6)

In [46]:
dx_spark = spark.createDataFrame(dx_feat.loc[:10,:])
rx_spark = spark.createDataFrame(rx_feat.loc[:10,:])
px_spark = spark.createDataFrame(px_feat.loc[:10,:])

dx_spark.createOrReplaceTempView('dx_spark_view')
rx_spark.createOrReplaceTempView('rx_spark_view')
px_spark.createOrReplaceTempView('px_spark_view')

In [47]:
%sql
drop table if exists dx_weights_chi2_new;

create table dx_weights_chi2_new as
select * from dx_spark_view;

drop table if exists rx_weights_chi2_new;

create table rx_weights_chi2_new as
select * from rx_spark_view;

drop table if exists px_weights_chi2_new;

create table px_weights_chi2_new as
select * from px_spark_view;

In [50]:
X = pd.concat([dx_feat, rx_feat, px_feat], axis=1)

In [51]:
feat_spark = spark.createDataFrame(X.loc[:1, :])
feat_spark.createOrReplaceTempView('feat_spark_view')

In [52]:
%sql

drop table if exists feat_weights_chi2;

create table feat_weights_chi2 as
select * from feat_spark_view;

## XGBoost

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix

In [56]:
import xgboost as xgb

def xgb_model(train_data, train_label, test_data, test_label):
    clf = xgb.XGBClassifier(max_depth=10,
                           min_child_weight=1,
                           njobs=16,
                           learning_rate=0.1,
                           n_estimators=150,
                           verbosity=3,
                           objective='binary:logistic',
                           gamma=0,
                           max_delta_step=0,
                           subsample=1,
                           colsample_bytree=1,
                           colsample_bylevel=1,
                           reg_alpha=0,
                           reg_lambda=0,
                           scale_pos_weight=1,
                           random_state=1,
                           missing=None)
    clf.fit(train_data, train_label, eval_metric='auc', verbose=True,
            eval_set=[(test_data, test_label)], early_stopping_rounds=5)
    y_pre = clf.predict(test_data)
    y_pro = clf.predict_proba(test_data)[:, 1]
    print("AUC Score : %f" % roc_auc_score(test_label, y_pro))
    print("Accuracy : %.4g" % accuracy_score(test_label, y_pre))
    return clf 

In [57]:
def fit_xgbClassifier(x, y):
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0, stratify=y)
  classifier = xgb_model(x_train, y_train, x_test, y_test)
  return classifier

In [59]:
dx_xgb = fit_xgbClassifier(df_dict['DX_sparse'], np.array(df_dict['DX_y']['switch_flag']))

[09:33:30] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 308 extra nodes, 0 pruned nodes, max_depth=10
[0]	validation_0-auc:0.6362
Will train until validation_0-auc hasn't improved in 5 rounds.
[09:33:31] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 324 extra nodes, 0 pruned nodes, max_depth=10
[1]	validation_0-auc:0.64413
[09:33:32] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 332 extra nodes, 0 pruned nodes, max_depth=10
[2]	validation_0-auc:0.645079
[09:33:32] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 362 extra nodes, 0 pruned nodes, max_depth=10
[3]	validation_0-auc:0.648312
[09:33:33] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 342 extra nodes, 0 pruned nodes, max_depth=10
[4]	validation_0-auc:0.648771
[09:33:33] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 326 extra nodes, 0 pruned nodes, max_depth=10
[5]	validation_0-auc:0.650799
[09:33:34] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 358 extra nodes, 0 pruned nodes, max_depth=10
[6]	validation_0-auc:0.651016
[09:33:34] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 326 extra nodes, 0 pruned nodes, max_depth=10
[7]	validation_0-auc:0.651377
[09:33:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 410 extra nodes, 0 pruned nodes, max_depth=10
[8]	validation_0-auc:0.651594
[09:33:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 400 extra nodes, 0 pruned nodes, max_depth=10
[9]	validation_0-auc:0.654503
[09:33:36] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 446 extra nodes, 0 pruned nodes, max_depth=10
[10]	validation_0-auc:0.655059
[09:33:36] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 474 extra nodes, 0 pruned nodes, max_depth=10
[11]	validation_0-auc:0.656654
[09:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 496 extra nodes, 0 pruned nodes, max_depth=10
[12]	validation_0-auc:0.656954
[09:33:38] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 472 extra nodes, 0 pruned nodes, max_depth=10
[13]	validation_0-auc:0.658407
[09:33:38] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 570 extra nodes, 0 pruned nodes, max_depth=10
[14]	validation_0-auc:0.65881
[09:33:39] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 540 extra nodes, 0 pruned nodes, max_depth=10
[15]	validation_0-auc:0.659382
[09:33:39] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 588 extra nodes, 0 pruned nodes, max_depth=10
[16]	validation_0-auc:0.660165
[09:33:40] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 584 extra nodes, 0 pruned nodes, max_depth=10
[17]	validation_0-auc:0.660888
[09:33:40] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 578 extra nodes, 0 pruned nodes, max_depth=10
[18]	validation_0-auc:0.661704
[09:33:41] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 554 extra nodes, 0 pruned nodes, max_depth=10
[19]	validation_0-auc:0.661638
[09:33:41] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 600 extra nodes, 0 pruned nodes, max_depth=10
[20]	validation_0-auc:0.661
[09:33:42] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 648 extra nodes, 0 pruned nodes, max_depth=10
[21]	validation_0-auc:0.660656
[09:33:43] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 640 extra nodes, 0 pruned nodes, max_depth=10
[22]	validation_0-auc:0.660978
[09:33:43] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 636 extra nodes, 0 pruned nodes, max_depth=10
[23]	validation_0-auc:0.660319
Stopping. Best iteration:
[18]	validation_0-auc:0.661704

AUC Score : 0.661704
Accuracy : 0.9887

In [60]:
rx_xgb = fit_xgbClassifier(df_dict['RX_sparse'], np.array(df_dict['RX_y']['switch_flag']))

[09:36:39] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 130 extra nodes, 0 pruned nodes, max_depth=10
[0]	validation_0-auc:0.592354
Will train until validation_0-auc hasn't improved in 5 rounds.
[09:36:39] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 134 extra nodes, 0 pruned nodes, max_depth=10
[1]	validation_0-auc:0.613998
[09:36:39] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 142 extra nodes, 0 pruned nodes, max_depth=10
[2]	validation_0-auc:0.613856
[09:36:39] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 140 extra nodes, 0 pruned nodes, max_depth=10
[3]	validation_0-auc:0.613832
[09:36:40] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 138 extra nodes, 0 pruned nodes, max_depth=10
[4]	validation_0-auc:0.60496
[09:36:40] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 136 extra nodes, 0 pruned nodes, max_depth=10
[5]	validation_0-auc:0.613943
[09:36:40] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 138 extra nodes, 0 pruned nodes, max_depth=10
[6]	validation_0-auc:0.613623
Stopping. Best iteration:
[1]	validation_0-auc:0.613998

AUC Score : 0.613998
Accuracy : 0.9972

In [61]:
px_xgb = fit_xgbClassifier(df_dict['PX_sparse'], np.array(df_dict['PX_y']['switch_flag']))

[09:37:21] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 0 pruned nodes, max_depth=10
[0]	validation_0-auc:0.532277
Will train until validation_0-auc hasn't improved in 5 rounds.
[09:37:21] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 0 pruned nodes, max_depth=10
[1]	validation_0-auc:0.546298
[09:37:21] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 0 pruned nodes, max_depth=10
[2]	validation_0-auc:0.553567
[09:37:21] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 0 pruned nodes, max_depth=10
[3]	validation_0-auc:0.53447
[09:37:21] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=10
[4]	validation_0-auc:0.538858
[09:37:21] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=10
[5]	validation_0-auc:0.545527
[09:37:21] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 0 pruned nodes, max_depth=10
[6]	validation_0-auc:0.54311
[09:37:21] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nodes, max_depth=10
[7]	validation_0-auc:0.53404
Stopping. Best iteration:
[2]	validation_0-auc:0.553567

AUC Score : 0.553567
Accuracy : 0.9695

In [62]:
dx_weights_xgb = pd.DataFrame.from_dict(dict(zip(df_dict['DX_freq'].columns[1:], dx_xgb.feature_importances_)), orient='index').sort_values(0, ascending=False)
rx_weights_xgb = pd.DataFrame.from_dict(dict(zip(df_dict['RX_freq'].columns[1:], rx_xgb.feature_importances_)), orient='index').sort_values(0, ascending=False)
px_weights_xgb = pd.DataFrame.from_dict(dict(zip(df_dict['PX_freq'].columns[1:], px_xgb.feature_importances_)), orient='index').sort_values(0, ascending=False)

In [63]:
rx_weights_lr.to_pickle('rx_weights_lr.pkl')
px_weights_lr.to_pickle('px_weights_lr.pkl')
dx_weights_lr.to_pickle('dx_weights_lr.pkl')

rx_weights_xgb.to_pickle('rx_weights_xgb.pkl')
px_weights_xgb.to_pickle('px_weights_xgb.pkl')
dx_weights_xgb.to_pickle('dx_weights_xgb.pkl')

In [64]:
rx_weights_lr = pd.read_pickle('rx_weights_lr.pkl')
px_weights_lr = pd.read_pickle('px_weights_lr.pkl')
dx_weights_lr = pd.read_pickle('dx_weights_lr.pkl')

rx_weights_xgb = pd.read_pickle('rx_weights_xgb.pkl')
px_weights_xgb = pd.read_pickle('px_weights_xgb.pkl')
dx_weights_xgb = pd.read_pickle('dx_weights_xgb.pkl')

In [65]:
def percent_change(weights_df):
  t = weights_df.cumsum()/weights_df.sum()
  t.rename(columns={0:'Feat_Importance'}, inplace=True)
  t['Feat_Importance_lag'] = t.shift(1)
  t['%change'] = (t['Feat_Importance'] - t['Feat_Importance_lag'])*100/t['Feat_Importance_lag']
  return t

In [66]:
rx_weights_lr = percent_change(rx_weights_lr)
px_weights_lr = percent_change(px_weights_lr)
dx_weights_lr = percent_change(dx_weights_lr)

In [67]:
dx_codes = list(dx_weights_lr[dx_weights_lr['%change']>=1].index)
rx_codes = list(rx_weights_lr[rx_weights_lr['%change']>=1].index)
px_codes = list(px_weights_lr[px_weights_lr['%change']>=1].index)

In [68]:
print('DX:')
print(dx_codes)
print('RX:')
print(rx_codes)
print('PX:')
print(px_codes)

DX:
['I70.0', 'I70.203', 'E78.9', 'E78.5', 'I25.5', 'E78.4', 'E78.2', 'I70.91', 'I67.2', 'I24.8', 'E11.9', 'E78.1', 'I70.293', 'G45.9', 'T46.6X5A', 'I71.4', 'I67.9', 'I69.351', 'I63.50', 'E13.9', 'I69.354', 'M79.7', 'E10.9', 'K72.00', 'E11.00', 'E11.69', 'I71.9', 'I69.30', 'K75.9', 'I70.209', 'E11.42', 'E10.29', 'E11.29', 'E11.59', 'E11.49', 'I63.40', 'E11.36', 'I69.398', 'E11.22', 'I25.82', 'E10.40', 'E11.319', 'I63.131', 'I69.90', 'E11.41', 'I23.6', 'E11.8', 'I67.82', 'I69.320', 'E11.65', 'I20.0', 'Z82.49', 'M62.89', 'E10.65', 'K75.81', 'I70.90', 'K75.4', 'I69.359', 'I63.9', 'E11.21', 'E10.649', 'I69.959', 'E10.8']
RX:
['357620', '284296', '284299', '284297', '275956', '284295', '284301', '418432', '273586', '418429', '324834', '273510', '298533', '222972', '418430', '418425', '298531', '227064', '418431', '275957', '273587', '324833', '298534', '273513', '418428', '418433', '298532', '273511', '274091', '418427']
PX:
['33510', '33514', '33511', '34802', '37218', 'C9602', '33512', '34826', '33521', 'C9606', '37232', '92934', '92938', '35390', '92943', '92973', '37231', '37223', '34825', 'C9607', 'C9608', '33522', '33513', '37226', '35301', '92921', 'C9605', '92924', '37227', '37216', '92929', '37234', '35600', '92925', '33536', 'C9604']

In [69]:
XGB_wts_spark = spark.createDataFrame(wts_df.reset_index(drop=False))
XGB_wts_spark.createOrReplaceTempView('XGB_wts_view')

In [70]:
%sql
drop table if exists XGB_wts;

CREATE table XGB_wts as
select * from XGB_wts_view

In [71]:
rx_weights_lr.reset_index(inplace=True)
dx_weights_lr.reset_index(inplace=True)
px_weights_lr.reset_index(inplace=True)

In [72]:
rx_weights_lr['%change'].fillna(100, inplace=True)
px_weights_lr['%change'].fillna(100, inplace=True)
dx_weights_lr['%change'].fillna(100, inplace=True)

In [73]:
rx_weights_lr.columns

Out[85]: Index(['index', 'Feat_Importance', 'Feat_Importance_lag', '%change'], dtype='object')

In [74]:
dx_spark = spark.createDataFrame(dx_weights_lr)
rx_spark = spark.createDataFrame(rx_weights_lr)
px_spark = spark.createDataFrame(px_weights_lr)

dx_spark.createOrReplaceTempView('lr_dx_wts_view')
px_spark.createOrReplaceTempView('lr_px_wts_view')
rx_spark.createOrReplaceTempView('lr_rx_wts_view')

In [75]:
%sql
drop table if exists lr_dx_wts;

CREATE table lr_dx_wts as
select * from lr_dx_wts_view;

drop table if exists lr_rx_wts;

CREATE table lr_rx_wts as
select * from lr_rx_wts_view;

drop table if exists lr_px_wts;

CREATE table lr_px_wts as
select * from lr_px_wts_view;

## Threshold, Recall, Precision

In [77]:
from scipy.optimize import minimize
import seaborn as sns
import matplotlib.pyplot as plt

In [78]:
from sklearn.metrics import jaccard_similarity_score

In [79]:
def get_jaccard_score(cut_off):
  temp=[]
  for i in range(len(y_pred)):
    if y_pred[i]>=cut_off:
      temp.append(1)
    else:
      temp.append(0)
  return -1 * jaccard_similarity_score(y_test,temp)

In [80]:
def get_f1_score(cut_off):
    temp = []
    for i in range(len(y_pred)):
        if y_pred[i]>=cut_off:
            temp.append(1)
        else:
            temp.append(0)
    return -1 * f1_score(y_test,temp)

In [81]:
def get_thresh_prec_rec(y_pred):
  thresh = minimize(get_f1_score, 0 , method='nelder-mead',options={'xtol': 1e-8, 'disp': True})

  temp = []
  for i in range(len(y_pred)):
      if y_pred[i]>=thresh.x[0]:
          temp.append(1)
      else:
          temp.append(0)
  
  cm = confusion_matrix(y_test,temp)
  
  print("Threshold:",thresh.x[0])
  print("Recall:",cm[1,1]*100/cm[1,0],'%')
  print("Precision:",cm[1,1]*100/cm[0,1],'%')
  
  return temp

In [82]:
def plot_confusion_matrix(y_test, final_y_pred):
  sns.heatmap(confusion_matrix(y_test,final_y_pred),annot=True)
  plt.ylabel('True Label')
  plt.xlabel('Predicted Label')
  plt.title('Confusion Matrix')
  display(plt.show())

In [83]:
final_y_pred = get_thresh_prec_rec(y_pred)

Optimization terminated successfully.
 Current function value: -0.090105
 Iterations: 29
 Function evaluations: 68
Threshold: 0.03531250000000001
Recall: 16.666666666666668 %
Precision: 7.044079515989629 %

In [84]:
plot_confusion_matrix(y_test, final_y_pred)